In [ ]:
import sys
import os
os.environ['OPP4SAR_DIR'] = '/home/jhewers/Documents/meng_project/code/'
sys.path.insert(0,os.getenv('OPP4SAR_DIR'))
DIR = os.path.join(os.getenv('OPP4SAR_DIR'),'generated_data/performance')

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rcParams.update({
    'axes.grid':False,
    'figure.dpi':100,
    'image.cmap':'gray',
    'legend.loc':'upper left',
    'legend.fancybox':True,
    'savefig.bbox':'tight'
    })

In [ ]:
import json
import numpy as np

from src.json_helpers import GlobalJsonDecoder
from src.enums import WaypointAlgorithmEnum

In [ ]:
N = 100
x,y = np.meshgrid(np.arange(0,N),np.arange(0,N))
x,y = x.flatten(),y.flatten()
xy = np.vstack((x,y)).T

z = np.ones(x.shape)
z = (z*100*np.random.random(x.shape)).astype(int)

pts = []

for f,g in zip(xy,z):
    [pts.append(f) for i in range(g)]
    
pts = np.array(pts)
print(f"{N**2}/{len(pts)} < 1 -> {N**2/len(pts)<1}")
np.testing.assert_equal(len(pts),np.sum(z))

In [ ]:
def non_duped(pts,N,R):
    pos = np.array([N//2,N//2])
    
    t = 0

    found = np.linalg.norm(pos-pts,axis=1)<R
    found_objs = []

    for f in pts[found]:
        found_objs.append((f,t))
    return found_objs

def duped(pts,N,R):
    pos = np.array([N//2,N//2])
    t = 0
    
    x,y = np.meshgrid(np.arange(0,N),np.arange(0,N))
    x,y = x.flatten(),y.flatten()
    xy = np.vstack((x,y)).T
    
    found = np.linalg.norm(pos-xy,axis=1)<R
    found_objs = []
    
    for f in xy[found]:
        found_objs.append((f,t))
    
    return found_objs

In [ ]:
%alias_magic t timeit -p "-n10 -o -q"

N = 500
x,y = np.meshgrid(np.arange(0,N),np.arange(0,N))
x,y = x.flatten(),y.flatten()
xy = np.vstack((x,y)).T

t1 = []
t2 = []
ratio = []
i_store = np.arange(0.6,14,0.1)

for i in i_store:
    print(f"{100*(i-np.min(i_store))/np.max(i_store):.2f}%")
    z = np.ones(x.shape)
    z = np.round(z*i*np.random.random(x.shape),0).astype(int)

    pts = []

    for f,g in zip(xy,z):
        [pts.append(f) for i in range(g)]

    pts = np.array(pts)
    np.testing.assert_equal(len(pts),np.sum(z))
    
    a = %t non_duped(pts,N,100)
    b = %t duped(pts,N,100) 
    
    t1.append(a.average)
    t2.append(b.average)
    ratio.append(N**2/np.sum(z))
    
print("Done")

In [ ]:
fig,ax1=plt.subplots()
a = ax1.plot(ratio,t1,label="Non-duped")
b = ax1.plot(ratio,t2,label="Duped")

mnx,mxx = ax1.get_xlim()
ax1.set_xlim((mxx,mnx))
ax1.plot([1,1],ax1.get_ylim(),'-.')
ax1.text(1,np.mean(ax1,get_ylim()),"$A/N_\text_{person} = 1$")

ax1.set_xlabel(r"$A/N_\text_{person}$")
ax1.set_ylabel(r"time ($s$)")

lns = a+b
plt.legend(lns,[l.get_label() for l in lns])


plt.show()